In [1]:
import pandas as pd
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from textblob import TextBlob
import wordsegment as ws 
ws.load()
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
data_technology = pd.read_json('tweets_technology.json', lines = True)
data_technology.sort_values('text',inplace=True)
data_technology.drop_duplicates(subset='text',keep = False, inplace=True)
data_technology = data_technology.sample(frac = 0.51)
data_technology.shape

(4699, 38)

In [3]:
def clean_tweets(df):
  i = 1  
  all_tweets = []
  tweets = df['text'].values.tolist();
  for text in tweets:
    print()
    print(i)
    i +=1
    print(text)
    text = text.lower()
    text_pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|))+')
    text = text_pattern.sub('', text)
    emoji = re.compile("["
                        u"\U0001F600-\U0001FFFF"   #EMOJIS
                        u"\U0001F300-\U0001F5FF"   #Symbols and Pictographs
                        u"\U0001F680-\U0001F6FF"   #map symbols
                        u"\U0001F1E0-\U0001F1FF"   #flags
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    text = emoji.sub(r'', text)
    text = re.sub(r'@[a-z0-9]+','',text)
   # text = re.sub(r'#', '', text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"this's", "this is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", "will", text)
    text = re.sub(r"\'ve", "have", text)
    text = re.sub(r"\'m", "am", text)
    text = re.sub(r"\'d", "would", text)
    text = re.sub(r"\'re", "are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"haven't", "have not", text)
    text = re.sub(r'rt[\s]+', '', text)

    text = re.sub(r"[,.\"!#$%^&*(){}?/;'~:<>+=-]", "", text)
    
    tokens = word_tokenize(text)
    
    text = [ws.clean(w) for w in text if len(ws.clean(w)) > 0]
    text = [ws.segment(w)[0] for w in text]   #segmentation
    
    stop_words = set(stopwords.words("english"))
    stop_words.discard("not")
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]

    words = [w for w in words if not w in stop_words]
    words = ' '.join(words)
        
    print(words)
    
    all_tweets.append(words)
  return all_tweets

In [4]:
nan_value = float("NaN")

Cleaned_tweets = clean_tweets(data_technology)

data_technology['text_cleaned'] = Cleaned_tweets

data_technology.replace("", nan_value, inplace=True)
data_technology.dropna(subset=['text_cleaned'], inplace=True)

data_technology = data_technology.sample(frac = 1)

data_technology = data_technology[data_technology['text_cleaned'].str.split().str.len().gt(4)]

data_technology = data_technology[['id', 'text_cleaned','text']]

data_technology.to_csv('tweets_cleaned_technology.csv',index=False)


1
RT @techjunkiejh: Here’s how neuroscience can protect AI from #cyberattacks https://t.co/OUyG6doGNq #artificalintelligence  #TechJunkieNews…
neuroscience protect ai cyberattacks artificalintelligence

2
@jongseongflirts @ENHYPEN_members Android

I vote for #ENHYPEN for #StanWorld 
#엔하이픈 @ENHYPEN @ENHYPEN_members
members android vote enhypen stanworld members

3
RT @wolfofpipst: BTC Daily Technical Outlook

Become a member and receive the calls. Profit with us. Link in bio.

#bitcoin #btc #cryptocur…
btc daily technical outlook become member receive calls profit us link bio bitcoin btc

4
Dumbest tweet of 2021 and it ain’t close
dumbest tweet close

5
#CryptoNews: Cryptocurrency exchange LATOKEN registered 100% growth in mobile app downloads in just two months, tha… https://t.co/nIQ9aKRz0R
cryptonews cryptocurrency exchange latoken registered growth mobile app downloads two months

6
RT @LF_Training: In this video, @linuxfoundation Training &amp; Certification instructor @TimSerewicz

authors predicted modern nearfuture world pkd kafka ballard mcluhan fill blank

100
Hahd
#etherium #bitcoin #cryptocurrency #crypto #blockchain #btc #bitcoinmining #coinbase #litecoin #ethereum… https://t.co/8Uk1vks2kT
hahd etherium bitcoin cryptocurrency crypto blockchain btc bitcoinmining coinbase litecoin

101
CRYPTO NEWS:  ‘Buy crypto’ #Google searches hit record high: The Tie https://t.co/9gPOmV6EDe #altcoin #altcoinnews… https://t.co/REUu0jESAl
crypto news buy crypto google searches hit record high tie altcoin

102
RT @techreview: [SPONSORED] Organizations are turning to AI to simplify and improve computer simulation. (With @siemenssoftware) https://t.…
sponsored organizations turning ai simplify improve computer simulation

103
Millennials: HIVE Stock Is a Much Better Buy Than Bitcoin https://t.co/fb5VTXfGJz
millennials hive stock much better buy bitcoin

104
1.4 million $DFN crypto tokens will be earned in the Difo Network airdrop on LATOKEN.
Get your share of rewards by… https

sadist kafka would say yes not us

177
RT @200Keks: WE WILL BEAT THIS FAT GUY 😂
beat fat guy

178
Read my InfoSecSherpa @Nuzzel Newsletter! Your guide up a mountain of information. #InfoSec #Cybersecurity… https://t.co/XMsbx0FIGW
read infosecsherpa newsletter guide mountain information infosec

179
Learn how to harness the power of data, AI, and automation to drive new business outcomes—register for FREE at Atmo… https://t.co/2ijTNFY2i2
learn harness power data ai automation drive new business free

180
30% OFF SALE begins NOW!!! 😄🛍️🔮 That's how you welcome Spring! 😉🌷

Use CODE: 'SPRING' at checkout to get 30% OFF, t… https://t.co/Naia3A8pu5
sale begins welcome spring use code spring checkout get

181
“Unhappiness is the result of self-sufficiency.”
- Generated from GPT-2 at https://t.co/CUNv0O9FqI
#machinewisdom… https://t.co/8wN0yNFize
unhappiness result selfsufficiency generated

182
🎓🌩️ #DevOps Engineer looking for a new challenge @Virtilabs are shaping the future of learning throu

top influencers future infrastructure vanloon

287
Enjoy Spring Break responsibly! 

Follow us at https://t.co/EzyDHKl8Js and download our new mobile app “C4 NOW” to… https://t.co/ePf4Gn5RQE
enjoy spring break responsibly follow us download new mobile app

288
great
great

289
RT @SLBriscoe: @suziday123 @Cbp8Cindy @cazij @DavidFoyn @_drazzari @joneill55 @jflorez @ja2cook @ccatmpt @pjervis @PoloOrtiz9 @drahus11 @si…
drazzari

290
RT @burkr_: I created a set of #iOS icon packs for #iPhone &amp; #iPad that you can get for FREE on my @buymeacoffee or @gumroad pages (link be…
created set ios icon packs iphone amp ipad get free pages link

291
Current Bitcoin Price:
USD $58,983.61
GBP £42,576.55
Euro €49,624.32
#bitcoin #btc $btc #btcusd #btcgbp #btceur #crypto #cryptocurrency
current bitcoin price usd gbp euro bitcoin btc btc btcusd btcgbp btceur crypto cryptocurrency

292
RT @sjvn: Android apps for Chromebooks: The essentials https://t.co/URpDmstRMP via @computerworld &amp; @JRRaphael #And

florida giving spring breaker whole new meaning

399
RT @cnaintelligence: The zones promise technological freedom, but what are the dangers to handing so much political power to big corporatio…
zones promise technological freedom dangers handing much political power big

400
People keep saying that the solution to gas is sharting. Can't we find a cleaner solution? https://t.co/8fB6jQfcVe… https://t.co/pNqrOpixvm
people keep saying solution gas sharting not find cleaner solution

401
RT @univ_rs: smells like spring up in here 🍃🍃
#GenshinImpact #原神 https://t.co/Y4nBwvwi84
rs smells like spring genshinimpact

402
We are so grateful to all users of our #opensource #database software! Thanks to you, some of them have just been r… https://t.co/bUfOuC1KnM
grateful users opensource database software thanks

403
RT @techjunkiejh: An introduction to data science and machine learning with Microsoft Excel https://t.co/PVE0MgGmYF #DataScience #DataMinin…
introduction data science machine learning m

...

la startup building tiny injectable robots attack tumors digital business tech covid

4215
RT @TheCryptoconomy: This is one of many reasons why simply saying...
"#Bitcoin uses X energy, therefore it is X amount of a bad thing" is…
one many reasons simply saying bitcoin uses x energy therefore x amount bad thing

4216
RT @ElectraProtocol: Fact 5⃣:

Sustainability is important for any project. At $XEP, there is ongoing non-stop #development activity by man…
fact sustainability important project xep ongoing nonstop development activity

4217
RT @FollettDeb: @CLSD_Schools Time to explore PA colleges and many out-of-state and international schools!  Register today!! https://t.co/y…
schools time explore pa colleges many outofstate international schools register today

4218
👨‍💻👩‍💻 Could you help be a Team Interview judge for the 2021 #SeaPerch Season? Get to know our teams and evaluate t… https://t.co/D1WhQ4EpK0
could help team interview judge seaperch season get know teams evaluate

4219
pls

trade crypto without spending money get assured share trade airdrop

4322
RT @ravenel_jeremy: The next generation of finance professionals is going to make a revolution...It gives me hope. Today I learned they alr…
jeremy next generation finance professionals going make revolutionit gives hope today learned

4323
Linux Fundamentals Part 2 - I have just completed this room! Check it out:  https://t.co/qzZZ8rHbFa #tryhackme… https://t.co/RR6RnpwMQE
linux fundamentals completed room check

4324
@Soccerzone4sho Bitcoin is real only when you meet the right account manager to instruct you and guide you
Since I… https://t.co/hxlgq0YjDw
bitcoin real meet right account manager instruct guide since

4325
My two favorite projects (and gainers) together today! $Epic $Vite
two favorite projects gainers together today epic vite

4326
RT @Androidheadline: Google Photos got a shiny new video editor with advanced features.
https://t.co/cFHhCCJuBP

#Google #GooglePhotos #And…
google photos got shiny new

consistent detection multiplets spectra using structure aware algorithms

4426
Me waiting for my ₹500 worth Bitcoin become ₹5 crore so I can quit my job: https://t.co/R5GXdTrNsu
waiting worth bitcoin become crore quit job

4427
RT @InvestorIdeas: #CBD #Stocks News: American Premium Water Corp. (OTC: $HIPH) Announces CBD Kit Launch Products for Cali Bear this Spring…
cbd stocks news american premium water corp otc hiph announces cbd kit launch products cali bear

4428
Pay attention #Bitcoin
pay attention bitcoin

4429
Start warming up！
stawarming

4430
RT @AttentionToken: Today we're announcing a new blockchain-based digital advertising platform based on a Basic Attention Token. https://t.…
today weare announcing new blockchainbased digital advertising platform based basic attention token

4431
Learn how to get started in #blockchain from the Women in Blockchain panel, hosted by Ripple Impact. Watch now.… https://t.co/UNpJgvUrhh
learn get started blockchain women blockchain panel hosted

spring street sidewalk sweeping commenced reminder parking prohibited city streets

4540
2 important #crypto updates:
🚀Michael Saylor: Apple, Google, or Facebook may get into #Bitcoin next
🚀Crypto exchang… https://t.co/UOdXN7VQUF
important crypto updates michael saylor apple google facebook may get bitcoin next crypto

4541
Traditional #hosting or #Cloud  – 
 Which is better for your business ?

https://t.co/hw5eu6eJ3w

#businessgrowth… https://t.co/GfFPYEMX0v
traditional hosting cloud better business

4542
It's March 19, 2021 at 04:14AM to buy a #oneplus #device #smoothos #90hz #oneplus8tpro #neversettle #beyondthespeed… https://t.co/8mSdJBZnJ8
march buy oneplus device smoothos neversettle

4543
RT @RegTechInsight: @CSSregtech is joining us at #RTSV, 16-17th June! 

Join the global #regtech community to explore the current #regulato…
joining us rtsv june join global regtech community explore current

4544
Go l’aider
go l aider

4545
RT @CoinMarketCap: Have you used our portfolio updat

good perspective cryptovisor regarding cardano stake pools

4652
RT @NttResearch: Have you read @KazuhiroGomi's latest @Forbes article on post-quantum #cryptography yet? Check it out here: https://t.co/Nl…
read latest article postquantum cryptography yet check

4653
Meet the #virtualreality cast ... my avatar

https://t.co/UChPp3W29w 

#animation #3danimation #animatedcharacters… https://t.co/il0FhWZMCn
meet virtualreality cast avatar animation

4654
RT @AchiChinelo: @CiscoDevNet I will develop mining bot trading bot bitcoin mining bot crypto
#bots #robots #chatbots #artificialintelligen…
develop mining bot trading bot bitcoin mining bot crypto bots robots chatbots

4655
👉 What is digital citizenship? Are you students aware? You can now participate in the #eTwinning Spring Campaign wi… https://t.co/II8IhT3GWE
digital citizenship students aware participate etwinning spring campaign

4656
Canadian Condo Company Invests in Bitcoin, Hopes to Become ‘Self-Sustaining Real Estate’ https://t.c